<a href="https://colab.research.google.com/github/TonatiuCG666/Chemioinformatic-Tools/blob/main/SMILES_to_SDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install rdkit


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, SDWriter
from google.colab import files

In [11]:
uploaded = files.upload()

Saving NotPeptides_1.csv to NotPeptides_1 (1).csv


In [12]:
# Read the CSV file
csv_filename = list(uploaded.keys())[0]  # Get the uploaded file name
data = pd.read_csv(csv_filename)

In [13]:
data.shape

(89, 3)

In [14]:
show_data = data.head(5)
show_data


,CompoundName,Compound_Class,SMILES
0,12-epi-hapalindole E,alkaloid,CC(=C)[C@H]1C[C@H]([C@@]([C@@H]([C@@H]1C2=CNC3...
1,Hapalindole A isothiocyanate,alkaloid,Cl[C@@H]1C[C@@H]2C(C)(C)C3=C(C([C@@H]2[C@@H](N...
2,Nostocine A,alkaloid,CN1N=CN=C2C1=NNC2=O
3,Tubercidin,alkaloid,C1=CN(C2=NC=NC(=C21)N)[C@H]3[C@@H]([C@@H]([C@H...
4,Glycine betaine,Amino acid derivative,C[N+](C)(C)CC(=O)[O-]


In [15]:
# Specify columns for SMILES and compound names
smiles_column = "SMILES"  # Column containing SMILES strings
name_column = "CompoundName"  # Column containing names of compounds

if smiles_column not in data.columns or name_column not in data.columns:
    raise ValueError(f"Columns '{smiles_column}' or '{name_column}' not found in the CSV file. Please check the column names.")


In [16]:
# Extract SMILES and names
smiles_list = data[smiles_column].dropna().tolist()
compound_names = data[name_column].dropna().tolist()


**CREAR ARCHIVOS SDF INDIVIDUALES**

In [ ]:
# Create a directory to save individual SDF files
output_dir = "CyanoMet"
os.makedirs(output_dir, exist_ok=True)

for i, (smi, compound_name) in enumerate(zip(smiles_list, compound_names)):
    try:
        mol = Chem.MolFromSmiles(smi)  # Convert SMILES to RDKit molecule
        if mol is None:
            print(f"Invalid SMILES: {smi}")
            continue

        # Generate 3D coordinates
        mol = Chem.AddHs(mol)  # Add hydrogens
        AllChem.EmbedMolecule(mol)  # Generate 3D coordinates
        AllChem.UFFOptimizeMolecule(mol)  # Optimize geometry

        # Set molecule name
        mol.SetProp("_Name", compound_name)  # Assign the compound name to the molecule

        # Replace invalid filename characters in compound name
        sanitized_name = "".join(c if c.isalnum() or c in "-_." else "_" for c in compound_name)

        # Save to an individual SDF file
        sdf_filename = os.path.join(output_dir, f"{sanitized_name}.sdf")
        with SDWriter(sdf_filename) as writer:
            writer.write(mol)

    except Exception as e:  # Catch specific exceptions if possible
        print(f"Error processing SMILES '{smi}' with name '{compound_name}': {e}")
        continue

# Compress the directory into a zip file for download
!zip -r CyanoMet.zip CyanoMet

# Download the zip file
files.download("CyanoMet.zip")

**CREAR UN SOLO ARCHIVO CON TODOS LOS SDF**

In [17]:
from rdkit import Chem
from rdkit.Chem import AllChem, SDWriter # Import SDWriter directly from rdkit.Chem
from google.colab import files

# Define the path for the output SDF file
output_file = "NonPpetides_2.sdf"

# Create an SDWriter that will write to the single SDF file
with SDWriter(output_file) as writer:
    for i, (smi, compound_name) in enumerate(zip(smiles_list, compound_names)):
        try:
            mol = Chem.MolFromSmiles(smi)  # Convert SMILES to RDKit molecule
            if mol is None:
                print(f"Invalid SMILES: {smi}")
                continue

            # Generate 3D coordinates
            mol = Chem.AddHs(mol)  # Add hydrogens
            AllChem.EmbedMolecule(mol)  # Generate 3D coordinates
            AllChem.UFFOptimizeMolecule(mol)  # Optimize geometry

            # Set molecule name
            mol.SetProp("_Name", compound_name)  # Assign the compound name to the molecule

            # Write molecule to the SDF file
            writer.write(mol)

        except Exception as e:  # Catch specific exceptions if possible
            print(f"Error processing SMILES '{smi}' with name '{compound_name}': {e}")
            continue

# Download the SDF file directly
files.download(output_file) # Use the output_file variable for the download

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>